# Imports

In [1]:
import numpy as np
import os
import sys
import pickle

## Helper Functions

In [2]:
%store -r analysis_hf
sys.path.insert(0, analysis_hf)
import zScore
import sexToBinary as sb
import wScore

# Set Directory Paths

In [3]:
# Directory path to load formatted Pathology Data
%store -r path_dataDir
# Directory path to load formatted Thickness Data
%store -r thick_dataDir
# Directory path to load formatted Thickness At Path Data
%store -r thickAtPath_dataDir
# Directory path to save Calcualted Analysis Data
%store -r thickAtPath_CalData
# Directory path to save Figures
%store -r thickAtPath_Fig

# Loading

## Loading Center of Mass of Regions (in this case it is CoM of Pathology regions)

In [4]:
with open(os.path.join(path_dataDir, 'pathCoM.pkl'), 'rb') as f:
    pathCoM = pickle.load(f)
f.close()

## Loading Raw Thickness/Volume Data

In [5]:
# HCResults
with open(os.path.join(thick_dataDir, 'HCResults.pkl'), 'rb') as f:
    HCResults = pickle.load(f)
f.close()

# PatientTAUResults
with open(os.path.join(thick_dataDir, 'PatientTAUResults.pkl'), 'rb') as f:
    PatientTAUResults = pickle.load(f)
f.close()

# PatientTDPResults
with open(os.path.join(thick_dataDir, 'PatientTDPResults.pkl'), 'rb') as f:
    PatientTDPResults = pickle.load(f)
f.close()

## Loading Thickness Values at Pathology Regions

In [6]:
# HCthicknessAtPath
with open(os.path.join(thickAtPath_dataDir, 'HCthicknessAtPath.pkl'), 'rb') as f:
    HCthicknessAtPath = pickle.load(f)
f.close()

# TAUthicknessAtPath
with open(os.path.join(thickAtPath_dataDir, 'TAUthicknessAtPath.pkl'), 'rb') as f:
    TAUthicknessAtPath = pickle.load(f)
f.close()

# TDPthicknessAtPath
with open(os.path.join(thickAtPath_dataDir, 'TDPthicknessAtPath.pkl'), 'rb') as f:
    TDPthicknessAtPath = pickle.load(f)
f.close()

In [7]:
# Sanity Check
assert(HCthicknessAtPath.shape == (54, 40))
assert(TAUthicknessAtPath.shape == (26, 40))
assert(TDPthicknessAtPath.shape == (30, 40))

## Loading Volume Values at Pathology Regions

In [8]:
# HCVolumeAtPath
with open(os.path.join(thickAtPath_dataDir, 'HCVolumeAtPath.pkl'), 'rb') as f:
    HCVolumeAtPath = pickle.load(f)
f.close()

# TAUVolumeAtPath
with open(os.path.join(thickAtPath_dataDir, 'TAUVolumeAtPath.pkl'), 'rb') as f:
    TAUVolumeAtPath = pickle.load(f)
f.close()

# TDPVolumeAtPath
with open(os.path.join(thickAtPath_dataDir, 'TDPVolumeAtPath.pkl'), 'rb') as f:
    TDPVolumeAtPath = pickle.load(f)
f.close()

In [9]:
# Sanity Check
assert(HCVolumeAtPath.shape == (54, 40))
assert(TAUVolumeAtPath.shape == (26, 40))
assert(TDPVolumeAtPath.shape == (30, 40))

# Formatting Data / Defining Variable

## Convert PathCoM to appropriate shape: 20 x 3 x 2 --> 40 x 3

In [10]:
pathCoM = np.vstack((pathCoM[:, :, 0], pathCoM[:, :, 1]))

## N = Number of regions we are analyzing for Thickness at Path

In [11]:
N = pathCoM.shape[0]
assert(N == 40)

# Calculating Z Score

## Z Score for Thickness Values

In [12]:
HCthicknessAtPath_z, TAUthicknessAtPath_z, TDPthicknessAtPath_z = zScore.generateZScore(HCthicknessAtPath, 
                                                                                        TAUthicknessAtPath, 
                                                                                        TDPthicknessAtPath)

## Z Score for Volume Values

In [13]:
HCVolumeAtPath_z, TAUVolumeAtPath_z, TDPVolumeAtPath_z = zScore.generateZScore(HCVolumeAtPath, 
                                                                               TAUVolumeAtPath, 
                                                                               TDPVolumeAtPath)

# Calculating W Score

## Parameters: Age, Sex, and ICV

## Get Age Values

In [14]:
# List of 54, 26, and 30 Denoting Age
ageHC = HCResults['Age'].reshape((-1, 1)) # (54,1)
ageTAU = PatientTAUResults['Age'].reshape((-1, 1)) # (26,1)
ageTDP = PatientTDPResults['Age'].reshape((-1, 1)) # (30,1)

assert(ageHC.shape == (HCthicknessAtPath.shape[0],1))
assert(ageTAU.shape == (TAUthicknessAtPath.shape[0],1))
assert(ageTDP.shape == (TDPthicknessAtPath.shape[0],1))

## Get Sex Values

In [15]:
# List of 54, 26, and 30 Denoting Sex
sexHC = HCResults['Sex'] # (54,1)
sexTAU = PatientTAUResults['Sex'] # (26,1)
sexTDP = PatientTDPResults['Sex'] # (30,1)

# Convert Male --> 0 / Female --> 1
sexHC = np.array(sb.sexToBinary(sexHC)).reshape((-1, 1)) #  (54,1)
sexTAU = np.array(sb.sexToBinary(sexTAU)).reshape((-1, 1)) #  (26,1)
sexTDP = np.array(sb.sexToBinary(sexTDP)).reshape((-1, 1)) #  (30,1)

assert(sexHC.shape == (HCthicknessAtPath.shape[0],1))
assert(sexTAU.shape == (TAUthicknessAtPath.shape[0],1))
assert(sexTDP.shape == (TDPthicknessAtPath.shape[0],1))

## Get ICV Values

In [16]:
icvHC = HCResults['Volume']['ICV'].reshape((-1, 1)) # (54,1)
icvTAU = PatientTAUResults['Volume']['ICV'].reshape((-1, 1)) # (26,1)
icvTDP = PatientTDPResults['Volume']['ICV'].reshape((-1, 1)) # (30,1)

assert(icvHC.shape == (HCthicknessAtPath.shape[0],1))
assert(icvTAU.shape == (TAUthicknessAtPath.shape[0],1))
assert(icvTDP.shape == (TDPthicknessAtPath.shape[0],1))

## Stack Age, Sex, and ICV values

In [17]:
# Stack the Age and Sex features into 2D array
hcCov = np.concatenate((ageHC, sexHC, icvHC), axis=1) # (54, 3)
tauCov = np.concatenate((ageTAU, sexTAU, icvTAU), axis=1)  # (26, 3)
tdpCov = np.concatenate((ageTDP, sexTDP, icvTDP), axis=1) # (30, 3)

assert(hcCov.shape == (HCthicknessAtPath.shape[0], 3))
assert(tauCov.shape == (TAUthicknessAtPath.shape[0], 3))
assert(tdpCov.shape == (TDPthicknessAtPath.shape[0], 3))

## Calculating W Score for Thickness Values

In [18]:
# Get W score for thickness values
HCthicknessAtPath_w, TAUthicknessAtPath_w, TDPthicknessAtPath_w = wScore.generateWScore(hcCov, tauCov, tdpCov, N, 
                                                                                        HCthicknessAtPath, 
                                                                                        TAUthicknessAtPath, 
                                                                                        TDPthicknessAtPath)

## Calculating W Score for Volume Values

In [19]:
# Get W score for thickness values
HCVolumeAtPath_w, TAUVolumeAtPath_w, TDPVolumeAtPath_w = wScore.generateWScore(hcCov, tauCov, tdpCov, N, 
                                                                               HCVolumeAtPath, 
                                                                               TAUVolumeAtPath, 
                                                                               TDPVolumeAtPath)

# Saving

## Save Calculated Z Scores

In [21]:
# HCthicknessAtPath_z
with open(os.path.join(thickAtPath_CalData, 'HCthicknessAtPath_z.pkl'), 'wb') as f:
    pickle.dump(HCthicknessAtPath_z, f)
f.close()

# TAUthicknessAtPath_z
with open(os.path.join(thickAtPath_CalData, 'TAUthicknessAtPath_z.pkl'), 'wb') as f:
    pickle.dump(TAUthicknessAtPath_z, f)
f.close()

# TDPthicknessAtPath_z
with open(os.path.join(thickAtPath_CalData, 'TDPthicknessAtPath_z.pkl'), 'wb') as f:
    pickle.dump(TDPthicknessAtPath_z, f)
f.close()

#-----

# HCVolumeAtPath_z
with open(os.path.join(thickAtPath_CalData, 'HCVolumeAtPath_z.pkl'), 'wb') as f:
    pickle.dump(HCVolumeAtPath_z, f)
f.close()

# TAUVolumeAtPath_z
with open(os.path.join(thickAtPath_CalData, 'TAUVolumeAtPath_z.pkl'), 'wb') as f:
    pickle.dump(TAUVolumeAtPath_z, f)
f.close()

# TDPVolumeAtPath_z
with open(os.path.join(thickAtPath_CalData, 'TDPVolumeAtPath_z.pkl'), 'wb') as f:
    pickle.dump(TDPVolumeAtPath_z, f)
f.close()

## Save Calcuated W Scores

In [22]:
# HCthicknessAtPath_w
with open(os.path.join(thickAtPath_CalData, 'HCthicknessAtPath_w.pkl'), 'wb') as f:
    pickle.dump(HCthicknessAtPath_w, f)
f.close()

# TAUthicknessAtPath_w
with open(os.path.join(thickAtPath_CalData, 'TAUthicknessAtPath_w.pkl'), 'wb') as f:
    pickle.dump(TAUthicknessAtPath_w, f)
f.close()

# TDPthicknessAtPath_w
with open(os.path.join(thickAtPath_CalData, 'TDPthicknessAtPath_w.pkl'), 'wb') as f:
    pickle.dump(TDPthicknessAtPath_w, f)
f.close()

#-----

# HCVolumeAtPath_w
with open(os.path.join(thickAtPath_CalData, 'HCVolumeAtPath_w.pkl'), 'wb') as f:
    pickle.dump(HCVolumeAtPath_w, f)
f.close()

# TAUVolumeAtPath_w
with open(os.path.join(thickAtPath_CalData, 'TAUVolumeAtPath_w.pkl'), 'wb') as f:
    pickle.dump(TAUVolumeAtPath_w, f)
f.close()

# TDPVolumeAtPath_w
with open(os.path.join(thickAtPath_CalData, 'TDPVolumeAtPath_w.pkl'), 'wb') as f:
    pickle.dump(TDPVolumeAtPath_w, f)
f.close()